<a href="https://colab.research.google.com/github/kabeerbora/1st_global/blob/main/arranging_asi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ASI Rearranging


In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
sheet_id = '1ziZc1OlSzIDhxmq1RtDbf_HQUHw_LPFoWQdoxuREc-4'
sheet_name = 'Sheet4'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df = pd.read_csv(url)

Mounted at /content/drive


In [ ]:
!pip install pyreadstat
import pandas as pd
from google.colab import drive
from glob import glob
import os
from tqdm.notebook import tqdm
import numpy as np
import geopandas as gpd
import subprocess
drive.mount("/content/drive")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 62.3 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
years = [f"{year}_{year+1}" for year in range(1999,2023)]

In [ ]:
%%time
#get spreadsheets key from url
gsheetkey = "1ogO9dkkgpr-kTcjYe__Yvw7-YiE6hFUXVAD2e-H2Olw"

#sheet name
sheet_name = 'Sheet1'

url=f'https://docs.google.com/spreadsheet/ccc?key={gsheetkey}&output=xlsx'
df_index = pd.read_excel(url,sheet_name=sheet_name, index_col=0)
df_index = df_index[~df_index['alias'].isna()]
# df_index['filename'] = df_index['filename'].str.strip().str.replace(r"[ ]+", "", regex=True)
df_index['year'] = df_index['year'].str.strip()
df_index = df_index[df_index["year"].isin(years)]
df_index.tail()

CPU times: user 1.88 s, sys: 73.6 ms, total: 1.95 s
Wall time: 4.47 s


,col_name,col_label,filename,year,alias,emp_labour,emp_labour_alias,sno,sno_values
4024,II6,Purchase value (in Rs.),blkI202223,2022_2023,pur_value_imp,NaN,NaN,NaN,NaN
4028,AJ01,"DSL (Block-A, Item 1)",blkJ202223,2022_2023,dsl,NaN,NaN,NaN,NaN
4029,J11,S. No. (to be entered as printed in the schedule),blkJ202223,2022_2023,sno,NaN,NaN,12.0,NaN
4034,J17,Gross sale value (Rs.),blkJ202223,2022_2023,gross_sale,NaN,NaN,NaN,NaN
4036,J19,"Excise Duty/ Sales Tax/VAT/ Other Taxes, ifany...",blkJ202223,2022_2023,excise_duty,NaN,NaN,NaN,NaN


In [ ]:
def filter_E_by_sno(df, sno, sno_alias):
    cols = [i for i in df.columns if i not in ['dsl', 'sno']]
    xdf = df[df['sno']==sno][['dsl']+cols]
    new_cols = [f"{c}_{sno_alias}" for c in cols]
    xdf.columns = ["dsl"] + new_cols
    return xdf

def generate_emp_lab_dict(emp_lab_i, emp_lab_alias):

    column_index = eval(f"[{emp_lab_i}]")
    e_alias = eval(f"[{emp_lab_alias}]")

    d = dict(zip(column_index, e_alias))
    return d

def pivot_emp_labour_data(df, emp_labour_dict):
    final_df = None

    for i, (sx,sv) in enumerate(emp_labour_dict.items()):
        xdf = filter_E_by_sno(df, sx, sv)
        if i==0:
            final_df = xdf
        else:
            final_df = pd.merge(
                left=final_df,
                right=xdf,
                on='dsl',
                how="right"
            )

    return final_df

In [ ]:
%%time

dir = r"/content/drive/MyDrive/data_economics_research/ASI_Firms"
dfs = {}

for (year,filename), dfx in tqdm(df_index.groupby(['year', 'filename'], sort=False)):
    filepath = os.path.join(dir, year, filename+".sav")

    if os.path.isfile(filepath):

        filename = os.path.basename(filepath).split('.')[0].strip()
        year = os.path.basename(os.path.dirname(filepath))

        if dfx.shape[0] > 0:

            column_labels = list(dfx['col_name'].values)
            new_col_labels = list(dfx['alias'].values)
            col_mapper = dict(zip(column_labels, new_col_labels))

            dfy = pd.read_spss(filepath, convert_categoricals=False, usecols=column_labels)
            dfy = dfy.rename(columns=col_mapper)

            ## Strip all column names and lower case them to maintain standardization
            dfy.columns = [c.strip().lower() for c in dfy.columns]

            if 'sno' in dfy.columns:
                dfy['sno'] = dfy['sno'].astype(int)


            if 'scheme_code' in dfy.columns:
                dfy['scheme_code'] = dfy['scheme_code'].astype(int)

                ## Filter census only data
                # dfy = dfy[dfy['scheme_code']==1]

            if "itm_code" in new_col_labels:
                dfy = dfy.loc[dfy.groupby("dsl")['itm_code'].transform("max")==dfy['itm_code']].copy()
                dfy.drop('itm_code', axis=1, inplace=True)

            sno_values = dfx.loc[(dfx['filename']==filename) & (~dfx['sno'].isna()), 'sno'].values

            ## Change the data type for serial number to Int:
            if len(sno_values)==1:
                sno = sno_values[0].astype(int)
            else:
                sno = None

            ## update metadata for each dataframe
            dfy.attrs['sno'] = sno
            dfy.attrs['filename'] = filename
            dfy.attrs['year'] = year

            ## Set the DSL data type to int
            dfy['dsl'] = dfy['dsl'].astype(int)

            ## Check if there are any additional notes for employment and labour

            if dfx['emp_labour'].notna().sum()==1:

                ## Then this is employee and labour data
                emp_labour_i = dfx.loc[dfx['alias']=='sno', "emp_labour"].iloc[0]
                emp_labour_v = dfx.loc[dfx['alias']=='sno', "emp_labour_alias"].iloc[0]

                ## Generate dict
                emp_labour_dict = generate_emp_lab_dict(emp_labour_i, emp_labour_v)

                ## Process employer and labour data
                dfy = pivot_emp_labour_data(dfy, emp_labour_dict)

            else:
                if "sno" in dfy.columns:
                    dfy = dfy.loc[(dfy['sno']==dfy.attrs['sno'])]

            ## Store the dataframes into respective years
            if year in dfs.keys():
                dfs[year].append(dfy)
            else:
                dfs.setdefault(year, [dfy])

  0%|          | 0/219 [00:00<?, ?it/s]

CPU times: user 1min 4s, sys: 4.71 s, total: 1min 9s
Wall time: 2min 40s


In [ ]:
from functools import reduce
import pandas as pd
# Define the range of years
years = [f"{year}_{year+1}" for year in range(1999, 2023)]

# Dictionary to store merged DataFrames for each year
merged_data = {}
for year in years:
    if year in dfs:  # Check if data exists for the given year
        dfz = dfs[year]  # Extract list of DataFrames for the year

        # Drop 'sno' from all DataFrames except the first one
        for i in range(1, len(dfz)):
            if 'sno' in dfz[i].columns:
                dfz[i] = dfz[i].drop(columns=['sno'])

        # Perform outer join across all DataFrames for the year
        df_merged = reduce(lambda left, right: pd.merge(left, right, on="dsl", how="outer"), dfz)

        # Add a 'year' column
        df_merged["year"] = year

        # Store merged DataFrame
        merged_data[year] = df_merged

# Concatenate all years into a single DataFrame
merged_df = pd.concat(merged_data.values(), ignore_index=True)
merged_df

,dsl,scheme_code,nic_code,state_code,district_code,sector_code,ownership,opening_gross_value,closing_gross_value,opening_net_value,...,year,rent_royalties_x,rent_rcvd_bldg,rent_royalties_y,int_received,rent_rcvd_land,gross_value,bonus,prov_fund,wel_exp
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1999_2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1999_2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1999_2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1999_2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1999_2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385517,219952,2.0,52109,36,99,2.0,NaN,NaN,NaN,NaN,...,2022_2023,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1385518,219953,2.0,52101,36,99,2.0,NaN,0.0,0.0,2814600.0,...,2022_2023,NaN,0.0,NaN,2390.0,0.0,NaN,0.0,0.0,0.0
1385519,219954,2.0,52101,36,99,1.0,NaN,0.0,0.0,11140810.0,...,2022_2023,NaN,0.0,NaN,3335.0,0.0,NaN,232000.0,0.0,10466.0
1385520,219955,2.0,18112,36,99,2.0,NaN,0.0,0.0,4537744.0,...,2022_2023,NaN,0.0,NaN,0.0,0.0,NaN,0.0,55980.0,38468.0


In [ ]:
num_dsls_below_10000 = len(merged_df[merged_df['dsl'] < 10000])
print(f"Number of DSLs below 10000: {num_dsls_below_10000}")
merged_df = merged_df[merged_df['dsl'] >= 10000]
print(f"Number of rows in merged_df after removal: {len(merged_df)}")
merged_df

Number of DSLs below 10000: 3486
Number of rows in merged_df after removal: 1382036


,dsl,scheme_code,nic_code,state_code,district_code,sector_code,ownership,opening_gross_value,closing_gross_value,opening_net_value,...,year,rent_royalties_x,rent_rcvd_bldg,rent_royalties_y,int_received,rent_rcvd_land,gross_value,bonus,prov_fund,wel_exp
3486,10001,1.0,24130.0,5,1.0,1.0,1.0,838612606.0,817821549.0,416737888.0,...,1999_2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3487,10002,1.0,15510.0,5,9.0,1.0,NaN,NaN,NaN,NaN,...,1999_2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3488,10003,1.0,15510.0,5,9.0,1.0,NaN,NaN,NaN,NaN,...,1999_2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3489,10004,1.0,15511.0,5,14.0,1.0,6.0,218370918.0,481196866.0,102261222.0,...,1999_2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3490,10005,1.0,15420.0,5,23.0,2.0,NaN,NaN,NaN,NaN,...,1999_2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385517,219952,2.0,52109,36,99,2.0,NaN,NaN,NaN,NaN,...,2022_2023,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1385518,219953,2.0,52101,36,99,2.0,NaN,0.0,0.0,2814600.0,...,2022_2023,NaN,0.0,NaN,2390.0,0.0,NaN,0.0,0.0,0.0
1385519,219954,2.0,52101,36,99,1.0,NaN,0.0,0.0,11140810.0,...,2022_2023,NaN,0.0,NaN,3335.0,0.0,NaN,232000.0,0.0,10466.0
1385520,219955,2.0,18112,36,99,2.0,NaN,0.0,0.0,4537744.0,...,2022_2023,NaN,0.0,NaN,0.0,0.0,NaN,0.0,55980.0,38468.0


In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
merged_df = merged_df.applymap(lambda x: str(x) if isinstance(x, float) else x)
merged_df.to_parquet("/content/drive/MyDrive/data_economics_research/ASI_Firms/1999_2023_all_firms.parquet")

,dsl,scheme_code,nic_code,state_code,district_code,sector_code,ownership,opening_gross_value,closing_gross_value,opening_net_value,...,year,rent_royalties_x,rent_rcvd_bldg,rent_royalties_y,int_received,rent_rcvd_land,gross_value,bonus,prov_fund,wel_exp
3486,10001,1.0,24130.0,5,1.0,1.0,1.0,838612606.0,817821549.0,416737888.0,...,1999_2000,nan,nan,nan,nan,nan,nan,nan,nan,nan
3487,10002,1.0,15510.0,5,9.0,1.0,nan,nan,nan,nan,...,1999_2000,nan,nan,nan,nan,nan,nan,nan,nan,nan
3488,10003,1.0,15510.0,5,9.0,1.0,nan,nan,nan,nan,...,1999_2000,nan,nan,nan,nan,nan,nan,nan,nan,nan
3489,10004,1.0,15511.0,5,14.0,1.0,6.0,218370918.0,481196866.0,102261222.0,...,1999_2000,nan,nan,nan,nan,nan,nan,nan,nan,nan
3490,10005,1.0,15420.0,5,23.0,2.0,nan,nan,nan,nan,...,1999_2000,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385517,219952,2.0,52109,36,99,2.0,nan,nan,nan,nan,...,2022_2023,nan,nan,nan,nan,nan,nan,0.0,0.0,0.0
1385518,219953,2.0,52101,36,99,2.0,nan,0.0,0.0,2814600.0,...,2022_2023,nan,0.0,nan,2390.0,0.0,nan,0.0,0.0,0.0
1385519,219954,2.0,52101,36,99,1.0,nan,0.0,0.0,11140810.0,...,2022_2023,nan,0.0,nan,3335.0,0.0,nan,232000.0,0.0,10466.0
1385520,219955,2.0,18112,36,99,2.0,nan,0.0,0.0,4537744.0,...,2022_2023,nan,0.0,nan,0.0,0.0,nan,0.0,55980.0,38468.0


In [ ]:
observations_by_year = merged_df.groupby('year').size()
observations_by_year

,0
year,
1999_2000,33515
2000_2001,41096
2001_2002,42242
2002_2003,41846
2003_2004,56889
2004_2005,49340
2005_2006,57304
2006_2007,66875
2007_2008,56888


In [ ]:
print("Number of dataframes per year:")
for year in years:
    print(year," | ",len(dfs[year]))
dfs

{'1999_2000': [         dsl  scheme_code  nic_code state_code  district_code  sector_code
  0      10001            1   24130.0          5            1.0          1.0
  1      10002            1   15510.0          5            9.0          1.0
  2      10003            1   15510.0          5            9.0          1.0
  3      10004            1   15511.0          5           14.0          1.0
  4      10005            1   15420.0          5           23.0          2.0
  ...      ...          ...       ...        ...            ...          ...
  33510  75817            2   93010.0         31            1.0          2.0
  33511  75818            2   93010.0         31            1.0          2.0
  33512  75819            2   93010.0         31            1.0          2.0
  33513  75820            2   93010.0         31            1.0          2.0
  33514  75821            2   93010.0         31            1.0          2.0
  
  [33515 rows x 6 columns],
           dsl  ownership
  0   

In [ ]:
years_to_keep = [f"{year}_{year+1}" for year in range(1999, 2010)]
df_new = merged_df[merged_df['year'].isin(years_to_keep)].copy()

# Shape File **matching**

In [ ]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/data_economics_research/ASI_Firms/district_codes_mm.xlsx')
df.sample(5)

,State,ec_district_code,asi_district,District Name,state_code_old,state_code,Dist_LGD,headquarter,latitude,longitude,formation_date,formation_details,PARENTDC
44,Punjab,8,10,Moga,3,3,38,Faridkot,30.6786,74.7558,1972,Formed from Firozpur district,10
109,Rajasthan,11,11,Dausa,8,8,97,Dausa,26.8900,76.3350,1991,Formed from Jaipur district,11
73,Haryana,6,6,Karnal,6,6,67,Karnal,29.6800,76.9800,1851,Formed from Delhi territory,6
618,Tamil Nadu,27,29,Tirunelveli,33,33,592,Tirunelveli,8.7300,77.6900,1790,Formed from parts of Madras Presidency,29
221,Bihar,19,19,Samastipur,10,10,217,Madhepura,25.9167,86.7833,1981,Created by splitting from Saharsa,19


In [ ]:
gdf = pd.read_excel('/content/drive/MyDrive/data_economics_research/ASI_Firms/gdf_districts.xlsx')
gdf_1 = gpd.read_file('/content/drive/MyDrive/data_economics_research/ASI_Firms/District Shape file/in_district_shp_files/in_district.shp')

In [ ]:
district_names_df = df['District Name'].str.lower().unique().tolist()
dtname_gdf = gdf_1['dtname'].str.lower().unique().tolist()
unmatched_districts = [district for district in district_names_df if district not in dtname_gdf]
print("Unmatched Districts:", unmatched_districts)

Unmatched Districts: ['almora']


## 1999 & 2000 ka milaap

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/data_economics_research/ASI_Firms/1999_2023_all_firms.parquet')

In [ ]:
years_to_keep = [1999,2000]
df_filtered = df[df['year'].isin(years_to_keep)].copy()
df_filtered

,dsl,scheme_code,nic_code,state_code,district_code,sector_code,ownership,opening_gross_value,closing_gross_value,opening_net_value,...,year,rent_royalties_x,rent_rcvd_bldg,rent_royalties_y,int_received,rent_rcvd_land,gross_value,bonus,prov_fund,wel_exp
3486,10001,1.0,24130.0,5,1.0,1.0,1.0,838612606.0,817821549.0,416737888.0,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3487,10002,1.0,15510.0,5,9.0,1.0,nan,nan,nan,nan,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3488,10003,1.0,15510.0,5,9.0,1.0,nan,nan,nan,nan,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3489,10004,1.0,15511.0,5,14.0,1.0,6.0,218370918.0,481196866.0,102261222.0,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3490,10005,1.0,15420.0,5,23.0,2.0,nan,nan,nan,nan,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78092,75282,2.0,24232.0,8,21.0,2.0,6.0,0.0,0.0,0.0,...,2000,0.0,0.0,0.0,0.0,nan,nan,nan,nan,nan
78093,75283,2.0,26960.0,8,30.0,2.0,6.0,nan,nan,nan,...,2000,nan,nan,nan,nan,nan,nan,nan,nan,nan
78094,75284,2.0,26960.0,8,30.0,2.0,3.0,0.0,0.0,0.0,...,2000,0.0,0.0,0.0,0.0,nan,nan,nan,nan,nan
78095,75285,2.0,15319.0,8,17.0,2.0,6.0,0.0,0.0,39500.0,...,2000,0.0,0.0,0.0,0.0,nan,nan,nan,nan,nan


In [ ]:
state_counts_by_year = df.groupby('year')['state_code'].nunique()
state_counts_by_year

,state_code
year,
1999_2000,28
2000_2001,31
2001_2002,31
2002_2003,31
2003_2004,31
2004_2005,31
2005_2006,31
2006_2007,31
2007_2008,31


In [ ]:
observations_by_state_year = []
for year, group in df.groupby('year'):
  state_counts = group['state_code'].value_counts()
  observations_by_state_year.append({year: state_counts.to_dict()})

observations_by_state_year

[{'1999_2000': {'23': 4231,
   '14': 3888,
   '7': 3163,
   '2': 2809,
   '25': 2530,
   '11': 2088,
   '26': 1842,
   '20': 1466,
   '13': 1404,
   '8': 1314,
   '21': 1268,
   '12': 1193,
   '5': 1026,
   '31': 1021,
   '19': 669,
   '4': 628,
   '30': 455,
   '29': 409,
   '6': 355,
   '33': 345,
   '9': 322,
   '24': 266,
   '10': 249,
   '28': 232,
   '18': 179,
   '15': 101,
   '16': 40,
   '27': 22}},
 {'2000_2001': {'33.0': 5719,
   '27.0': 4944,
   '24.0': 4111,
   '28.0': 3299,
   '9.0': 3096,
   '29.0': 2546,
   '19.0': 2042,
   '3.0': 1886,
   '6.0': 1655,
   '8.0': 1435,
   '32.0': 1373,
   '23.0': 1304,
   '7.0': 1126,
   '18.0': 819,
   '21.0': 693,
   '10.0': 630,
   '20.0': 625,
   '22.0': 605,
   '5.0': 475,
   '25.0': 410,
   '26.0': 383,
   '2.0': 343,
   '30.0': 315,
   '34.0': 307,
   '16.0': 260,
   '1.0': 236,
   '4.0': 176,
   '13.0': 164,
   '14.0': 65,
   '17.0': 31,
   '35.0': 23}},
 {'2001_2002': {'33.0': 5945,
   '27.0': 4562,
   '24.0': 3858,
   '28.0': 3

In [ ]:
df['state_code'] = pd.to_numeric(df['state_code'], errors='coerce').astype('Int64')
df['year'] = df['year'].astype(str).str[:4].astype(int)
all_states = set(range(1, 36))
state_codes_by_year = df.groupby('year')['state_code'].unique()
df
missing_states_by_year = {}

for year in range(1999, 2024):
    observed_states = set(state_codes_by_year.get(year, []))
    missing_states = all_states - observed_states
    missing_states_by_year[year] = sorted(missing_states)


missing_states_df = pd.DataFrame.from_dict(missing_states_by_year, orient='index')
missing_states_df.columns = [f"Missing_State_{i+1}" for i in range(missing_states_df.shape[1])]

missing_states_df

,dsl,scheme_code,nic_code,state_code,district_code,sector_code,ownership,opening_gross_value,closing_gross_value,opening_net_value,...,year,rent_royalties_x,rent_rcvd_bldg,rent_royalties_y,int_received,rent_rcvd_land,gross_value,bonus,prov_fund,wel_exp
3486,10001,1.0,24130.0,5,1.0,1.0,1.0,838612606.0,817821549.0,416737888.0,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3487,10002,1.0,15510.0,5,9.0,1.0,nan,nan,nan,nan,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3488,10003,1.0,15510.0,5,9.0,1.0,nan,nan,nan,nan,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3489,10004,1.0,15511.0,5,14.0,1.0,6.0,218370918.0,481196866.0,102261222.0,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3490,10005,1.0,15420.0,5,23.0,2.0,nan,nan,nan,nan,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385517,219952,2.0,52109,36,99,2.0,nan,nan,nan,nan,...,2022,nan,nan,nan,nan,nan,nan,0.0,0.0,0.0
1385518,219953,2.0,52101,36,99,2.0,nan,0.0,0.0,2814600.0,...,2022,nan,0.0,nan,2390.0,0.0,nan,0.0,0.0,0.0
1385519,219954,2.0,52101,36,99,1.0,nan,0.0,0.0,11140810.0,...,2022,nan,0.0,nan,3335.0,0.0,nan,232000.0,0.0,10466.0
1385520,219955,2.0,18112,36,99,2.0,nan,0.0,0.0,4537744.0,...,2022,nan,0.0,nan,0.0,0.0,nan,0.0,55980.0,38468.0


In [ ]:
df['state_code'] = pd.to_numeric(df['state_code'], errors='coerce').astype('Int64')
df['year'] = df['year'].astype(str).str[:4].astype(int)
df_filtered = df[(df['year'] == 1999) & (df['state_code'] == 5)]
num_unique_districts = df_filtered['district_code'].nunique()
num_unique_districts
df_filtered_10 = df[(df['year'] == 2000) & (df['state_code'] == 10)]
num_unique_districts = df_filtered_10['district_code'].nunique()
num_unique_districts

print(f"Number of unique district codes in 1999_2000 for state code 5: {num_unique_districts}")

30

In [ ]:
!pip install tabula-py
import tabula
tables = tabula.read_pdf("/content/drive/MyDrive/data_economics_research/ASI_Firms/ASISTATE_CODES.pdf", pages="all")
for i, table in enumerate(tables):
    print(f"Table {i+1}:")
table

Table 1:


,Sr.\rNo.,Name of State,State Code,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,1974-75\rTO\r1988-89,1989-90\rTO\r1990-91,1991-92\rTO\r1994-95,1995-96\rTO\r1999-00,2000-01\rto\r2002-03,NaN
1,1.0,ANDHRA PRADESH,41,02,02,02,28.0
2,2.0,ARUNCHAL PRADESH,39,N.A.-,N.A.,03,12.0
3,3.0,ASSAM,34,03,04,04,18.0
4,4.0,BIHAR,31,04,05,05,10.0
5,5.0,GOA,54,29,06,06,30.0
6,6.0,GUJARAT,51,05,07,07,24.0
7,7.0,HARYANA,14,06,08,08,6.0
8,8.0,HIMACHAL PRADESH,15,07,09,09,2.0
9,9.0,JAMMU & KASHMIR,13,08,10,10,1.0


In [ ]:
table = table.iloc[1:]
table

,Sr.\rNo.,Name of State,State Code,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
1,1.0,ANDHRA PRADESH,41,02,02,02,28.0
2,2.0,ARUNCHAL PRADESH,39,N.A.-,N.A.,03,12.0
3,3.0,ASSAM,34,03,04,04,18.0
4,4.0,BIHAR,31,04,05,05,10.0
5,5.0,GOA,54,29,06,06,30.0
6,6.0,GUJARAT,51,05,07,07,24.0
7,7.0,HARYANA,14,06,08,08,6.0
8,8.0,HIMACHAL PRADESH,15,07,09,09,2.0
9,9.0,JAMMU & KASHMIR,13,08,10,10,1.0
10,10.0,KARNATAKA,53,09,11,11,29.0


In [ ]:
table = table.rename(columns={
    'State  Code': '1974-75\rTO\r1988-89',
    'Unnamed: 0': '1989-90\rTO\r1990-91',
    'Unnamed: 1': '1991-92\rTO\r1994-95',
    'Unnamed: 2': '1995-96\rTO\r1999-00',
    'Unnamed: 3': '2000-01\rto\r2002-03'
})
table

,Sr.\rNo.,Name of State,1974-75\rTO\r1988-89,1989-90\rTO\r1990-91,1991-92\rTO\r1994-95,1995-96\rTO\r1999-00,2000-01\rto\r2002-03
1,1.0,ANDHRA PRADESH,41,02,02,02,28.0
2,2.0,ARUNCHAL PRADESH,39,N.A.-,N.A.,03,12.0
3,3.0,ASSAM,34,03,04,04,18.0
4,4.0,BIHAR,31,04,05,05,10.0
5,5.0,GOA,54,29,06,06,30.0
6,6.0,GUJARAT,51,05,07,07,24.0
7,7.0,HARYANA,14,06,08,08,6.0
8,8.0,HIMACHAL PRADESH,15,07,09,09,2.0
9,9.0,JAMMU & KASHMIR,13,08,10,10,1.0
10,10.0,KARNATAKA,53,09,11,11,29.0


In [ ]:
state_mapping = table_cleaned[['1995-96\rTO\r1999-00', '2000-01\rto\r2002-03']]
state_mapping = state_mapping.dropna()
state_mapping_dict
state_mapping_dict = state_mapping.set_index('1995-96\rTO\r1999-00')['2000-01\rto\r2002-03'].apply(lambda x: int(x)).to_dict()
df['state_code'] = pd.to_numeric(df['state_code'], errors='coerce')
df.loc[df['year'] == 1999, 'state_code'] = df.loc[df['year'] == 1999, 'state_code'].map(state_mapping_dict)

,dsl,scheme_code,nic_code,state_code,district_code,sector_code,ownership,opening_gross_value,closing_gross_value,opening_net_value,...,year,rent_royalties_x,rent_rcvd_bldg,rent_royalties_y,int_received,rent_rcvd_land,gross_value,bonus,prov_fund,wel_exp
3486,10001,1.0,24130.0,10.0,1.0,1.0,1.0,838612606.0,817821549.0,416737888.0,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3487,10002,1.0,15510.0,10.0,9.0,1.0,nan,nan,nan,nan,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3488,10003,1.0,15510.0,10.0,9.0,1.0,nan,nan,nan,nan,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3489,10004,1.0,15511.0,10.0,14.0,1.0,6.0,218370918.0,481196866.0,102261222.0,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
3490,10005,1.0,15420.0,10.0,23.0,2.0,nan,nan,nan,nan,...,1999,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385517,219952,2.0,52109,36.0,99,2.0,nan,nan,nan,nan,...,2022,nan,nan,nan,nan,nan,nan,0.0,0.0,0.0
1385518,219953,2.0,52101,36.0,99,2.0,nan,0.0,0.0,2814600.0,...,2022,nan,0.0,nan,2390.0,0.0,nan,0.0,0.0,0.0
1385519,219954,2.0,52101,36.0,99,1.0,nan,0.0,0.0,11140810.0,...,2022,nan,0.0,nan,3335.0,0.0,nan,232000.0,0.0,10466.0
1385520,219955,2.0,18112,36.0,99,2.0,nan,0.0,0.0,4537744.0,...,2022,nan,0.0,nan,0.0,0.0,nan,0.0,55980.0,38468.0


In [ ]:
df.to_parquet("/content/drive/MyDrive/data_economics_research/ASI_Firms/1999_2023_all_firms.parquet", index=False)

In [ ]:
observations_by_year = df.groupby('year').size()
observations_by_year

,0
year,
1999,33515
2000,41096
2001,42242
2002,41846
2003,56889
2004,49340
2005,57304
2006,66875
2007,56888


In [6]:
import pandas as pd
df = pd.read_stata('/content/drive/MyDrive/data_economics_research/ASI_Firms/District codes & names_nodup.dta')


,state_ORIGINAL,district,districtname,year,survey
0,A & N ISLANDS,1,Andaman,1989.0,ASI
1,A & N ISLANDS,2,Nicobar,1989.0,ASI
2,A & N ISLANDS,1,Andaman,1994.0,ASI
3,A & N ISLANDS,2,Nicobar,1994.0,ASI
4,A & N ISLANDS,1,Andaman,1999.0,ASI
...,...,...,...,...,...
7543,WEST BENGAL,15,Medinipur,2008.0,ASI
7544,WEST BENGAL,16,Howrah,2008.0,ASI
7545,WEST BENGAL,17,Kolkata,2008.0,ASI
7546,WEST BENGAL,18,South 24-Parganas,2008.0,ASI


In [9]:
# prompt: using df, list the the names of state_ORIGINAL in the year 1999.0

df_1999 = df[df['year'] == 1999.0]
state_names_1999 = df_1999['state_ORIGINAL'].unique()
state_names_1999



array(['A & N ISLANDS', 'ANDHRA PRADESH', 'ARUNACHAL PRADESH', 'ASSAM',
       'BIHAR', 'CHANDIGARH', 'DADRA & NAGAR HAVELI', 'DAMAN & DIU',
       'DELHI', 'GOA', 'GUJARAT', 'HARYANA', 'HIMACHAL PRADESH',
       'JAMMU & KASHMIR', 'KARNATAKA', 'KERALA', 'LAKSHADWEEP',
       'MADHYA PRADESH', 'MAHARASHTRA', 'MANIPUR', 'MEGHALAYA', 'MIZORAM',
       'NAGALAND', 'ORISSA', 'PONDICHERRY', 'PUNJAB', 'RAJASTHAN',
       'SIKKIM', 'TAMIL NADU', 'TRIPURA', 'UTTAR PRADESH', 'WEST BENGAL'],
      dtype=object)

In [4]:
df = pd.read_parquet('/content/drive/MyDrive/data_economics_research/ASI_Firms/observations_and_asi_merged_data.parquet')
df.columns

Index(['index', 'dsl', 'scheme_code', 'nic_code', 'state_code',
       'district_code', 'sector_code', 'ownership', 'opening_gross_value',
       'closing_gross_value', 'opening_net_value', 'closing_net_value',
       'man_total_C', 'wages_total_C', 'bonus_C', 'prov_fund_C', 'wel_exp_C',
       'man_total_TW', 'wages_total_TW', 'bonus_TW', 'prov_fund_TW',
       'wel_exp_TW', 'man_total_SM', 'wages_total_SM', 'bonus_SM',
       'prov_fund_SM', 'wel_exp_SM', 'man_total_T', 'wages_total_T', 'bonus_T',
       'prov_fund_T', 'wel_exp_T', 'op_expenses', 'total_exp',
       'rent_paid_bldg', 'rent_paid_land', 'int_paid', 'tot_receipts',
       'pur_value_ind', 'pur_value_imp', 'gross_sale', 'excise_duty',
       'sales_tax', 'others_product', 'total', 'year', 'rent_royalties_x',
       'rent_rcvd_bldg', 'rent_royalties_y', 'int_received', 'rent_rcvd_land',
       'gross_value', 'bonus', 'prov_fund', 'wel_exp', 'State',
       'ec_district_code', 'asi_district', 'PARENTDC', 'District Name',
 